# Ingest races.csv file

In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions

In [0]:
dbutils.widgets.text('p_data_source', '')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

# Step 1.- Read the CSV file using the spark dataframe reader

Del comando anterior, observamos que la columna de los titulos los toma como si fueran datos, para ello, tenemos que colocar un comando para que considere esa columna como el titulo de las columnas.

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
 #Con esto podemos generar nuesto propio schema a partir de los datos.

In [0]:
races_schema = StructType(fields=[StructField('raceId', IntegerType(), False),
                                    StructField('year', IntegerType(), True),
                                    StructField('round', IntegerType(), True),
                                    StructField('circuitId', IntegerType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('date', DateType(), True),
                                    StructField('time', StringType(), False),
                                    StructField('url', StringType(), True)
 ])

In [0]:
races_df = spark.read \
    .option('header', True) \
    .schema(races_schema) \
    .csv(f'{raw_folder_path}/{v_file_date}/races.csv')  #''schema(circuits_schema)'' le estamos diciendo que utilice el schema anterior generado por nosotros. Puede tambien ir otra opcion que es ''option('inferSchema', True)'' con esto le decimos que vaya a fondo en los datos y que nos diga su tipo de schema.


### Step 2.- Add ingestion date and race_timestamp to the dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, lit, col

In [0]:
races_with_timestamp_df = add_ingestion_date(races_df) \
                                .withColumn('race_timestamp', to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
                                .withColumn('data_source', lit(v_data_source)) \
                                .withColumn('file_date', lit(v_file_date))

In [0]:
display(races_with_timestamp_df)

## 3.- Select columns required

In [0]:
from pyspark.sql.functions import col 

In [0]:
races_selected_df = races_with_timestamp_df.select(col('raceId'), col('year'), col('round'), col('circuitId'), col('name'), col('ingestion_date'), col('race_timestamp'), col('data_source'))

### Step 4.- Rename the columns as required

In [0]:
races_renamed_df = races_selected_df.withColumnRenamed('raceId', 'race_id') \
  .withColumnRenamed('year', 'race_year') \
  .withColumnRenamed('circuitId', 'circuit_id')

## Step 5.- Write Data to DataLake as parquet

In [0]:
races_renamed_df.write.mode('overwrite').format('delta').saveAsTable('f1_processed.races') #Con la funcion df.write.parquet() le especificamos que nuestro dataframe se guarde como archivo parquet en la ruta mencionada.


In [0]:
%sql 
select * from f1_processed.races;

## Step 6.- Partition by year

In [0]:
races_renamed_df.write.mode('overwrite').option('overwriteSchema', 'true').partitionBy('race_year').format('delta').save(f'{processed_folder_path}/races')

In [0]:
dbutils.notebook.exit('Success')